# CTFAR-10
本次作业为CIFAR-10数据分类的程序设计问题。作业均采用Pytorch软件和Jupyter notebook 软件完成，上交文件命名为Homework2_姓名_学号.ipynb。助教会运行3次代码，取三次分类精度最高值为模型的最后精度，最后分类精度到达95%即完成。作业详细描述如下：
数据在共享文件Cifar10Dataset文件夹下，data_batch_1--data_batch_5文件为50000张训练样本图片，test_batch文件为10000张测试样本图片，更加详细的说明文档见数据包中的readme.html。建立Homework2_姓名_学号.ipynb文件，采用PyTorch中torchvision包的datasets.CIFAR10和torch.utils.data.DataLoader函数进行数据集读取。采用深度学习模型进行分类，模型中至少包括3层神经网络，分类精度依据测试集的表现为准。
完成的作业打包成压缩包，命名为”学号_姓名_学院_班级”，发送至邮箱1004848808@qq.com,作业提交时间：2022年1月7日 24:00。

## 进行深度学习
这里是使用resnet模型

In [1]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()

        #这里的即为两个3*3 conv
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),      #bias为偏置，False表示不添加偏置
            nn.BatchNorm2d(outchannel),
            nn.ReLU(),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential() #shortcut connections
        if stride != 1 or inchannel != outchannel: #判断入通道和出通道是否一样，不一样的话进行卷积操作
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock):
        super(ResNet, self).__init__()

        #图片处理，也就是白色方框内的3*3 conv
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

        #中间的残差网络部分，与图上的结构一一对应
        self.layer1 = self.make_layer(ResidualBlock, 64, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 64, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 128, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 256, 512, 2, stride=2)
        self.avg_pool2d = nn.AvgPool2d(4)
        self.fc = nn.Linear(512, 10)

    #相当于看处理几次，18的是每个处理两次
    def make_layer(self, block, inchannel, outchannel, num_blocks, stride):
        layers = []
        for i in range(1, num_blocks):
            layers.append(block(inchannel, outchannel, stride))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool2d(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet_cifar():
    return ResNet(ResidualBlock)


In [15]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
#from resnet import ResNet_cifar
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

num_epoch = 10
BATCH_SIZE = 20

transform = transforms.Compose(
    [transforms.ToTensor()
    ]
)

#MNIST数据集加载

train_dataset = datasets.CIFAR10(
    root= './data',
    train= True, 
    transform= transform, 
    download= True
)
train_loader = data.DataLoader(
    dataset= train_dataset, 
    batch_size= BATCH_SIZE, 
    shuffle= True
)
test_dataset = datasets.CIFAR10(
    root= './data', 
    train= False,
    transform= transform,
    download= True
)
test_loader = data.DataLoader(
    dataset= test_dataset, 
    batch_size= BATCH_SIZE, 
    shuffle= True
)

#定义网络
net = ResNet_cifar()
print(net)

#进行优化
#optimizer = torch.optim.RMSprop(net_cifar10.parameters(), lr = 0.005, alpha= 0.9)
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001, betas= (0.9, 0.99))
loss_function = nn.CrossEntropyLoss()

for epoch in range(num_epoch):
    print('epoch = %d' % epoch)
    for i, (image, label) in enumerate(train_loader):

        x = net(image)
        loss = loss_function(x, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print('loss = %.5f' % loss)

#对测试集的评估
total = 0
correct = 0
net.eval()

for image, label in test_loader:
    x = net(image)
    _, prediction = torch.max(x, 1)
    total += label.size(0)
    correct += (prediction == label).sum()
print('There are ' + str(correct.item()) + ' correct pictures.')
print('Accuracy=%.2f' % (100.00 * correct.item() / total))


Using downloaded and verified file: ./data/cifar-10-python.tar.gz
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
      (relu): ReLU()
    )
  )
  (layer2): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): 

## 结果
最终的精度为**82.62**

## 遇到的困难及解决方法
* **数据输入的万般尝试**

**问题**：
* 存在官网下载慢容易卡的问题，总是下载失败

**解决尝试**：
1. 本地下载数据，解压数据，企图使用numpy读取数据以替代torchvision的读取：

```Python
import numpy as np
import os

# cifar10数据导入函数

# 对每一个batch进行导入
def load_batch(file):
    import pickle
    with open(file, 'rb') as fo:                                         # 打开文件
        d = pickle.load(fo, encoding='bytes')                            # 导入文件，以bytes编码格式
        d_decoded = {}                                                   # 定义字典
        for k, v in d.items():                                           # 对数据集中的特征与标签遍历
            d_decoded[k.decode('utf8')] = v                              # 用数据集填充d_decoded
        d = d_decoded
        data = d['data']                                                 # 读出batch中所有特征存入data
        labels = d['labels']                                             # 读出batch中所有标签存入labels
        data = data.reshape(data.shape[0], 3, 32, 32)                    # 数组形变 
    return data, labels                                                  # 返回最原始的特征与标签   

def load_data(path ='cifar-10-batches-py'):
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    
    # 训练集训练数据数量
    num_train_samples = 50000
    
    # 新建数组变量，根据数据集情况建立，50000张3通道的32*32的图片
    x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.empty((num_train_samples,), dtype='uint8')

    # 这里一共有5组data_batch，1到5循环遍历，每一个batch有10000张图片，导入图片
    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))          # 根据文件夹进行绝对地址访问
        (x_train[(i - 1) * 10000: i * 10000, :, :, :],              #1-10000，10001-20000，20001-30000 等等    3通道32*32的彩色图
         y_train[(i - 1) * 10000: i * 10000]) = load_batch(fpath)
        
    fpath = os.path.join(path, 'test_batch')
    x_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))   # 增加一维,相当于把行向量变成列向量
    y_test = np.reshape(y_test, (len(y_test), 1))
    
    return (x_train, y_train), (x_test, y_test)


(x_train, y_train), (x_test, y_test) = load_data('cifar-10-batches-py')

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

x_test=x_test.transpose(0,2,3,1).astype("float")# 将3,32，32类型的数据转化为（32，32，3）
x_train=x_train.transpose(0,2,3,1).astype("float")

```

<img src="1.png" />

之后进行过将train_dataset，test_dataset构造tensor类型的元组的尝试……均以失败告终

2. 本地下载数据，更改root
   尝试过修改相对路径，更改文件名称等，均以在文件中找不到对应文件报错告终

**解决方法**：

参考连接：https://www.freesion.com/article/5198895969/

思路：下载本地数据，最后更改cifar.py文件中的url,使得“下载”直接从本地开始下载，即直接从本地进行导入

<img src="2.png" />

最后成功导入数据，进行后面的操作
